In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# 基于UNet3d进行改进

## 1. 网络结构

In [3]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
         )
    def forward(self, x):
        return self.conv(x)

### 1.1 UNet_BN

In [4]:
class UNet3d_bn(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet3d_bn, self).__init__()
        self.encoder1 = DoubleConv(in_channels, 32)
        self.encoder2 = DoubleConv(32, 64)
        self.encoder3 = DoubleConv(64, 128)
        self.encoder4 = DoubleConv(128, 256)
        self.encoder5 = DoubleConv(256, 512) 

        self.decoder1 = DoubleConv(512, 256)
        self.conv_trans1 = nn.ConvTranspose3d(512, 256, kernel_size=2, stride=2)
        self.decoder2 = DoubleConv(256, 128)
        self.conv_trans2 = nn.ConvTranspose3d(256, 128, kernel_size=2, stride=2)
        self.decoder3 = DoubleConv(128, 64)
        self.conv_trans3 = nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2)
        self.decoder4 = DoubleConv(64, 32)
        self.conv_trans4 = nn.ConvTranspose3d(64, 32, kernel_size=2, stride=2)
        self.out_conv = DoubleConv(32, out_channels)

        self.soft = nn.Softmax(dim=1)
        
        
    def forward(self, x):
        # 编码器部分
        t1 = self.encoder1(x)                                               # 32 x 128 x 128 x 128
        out = F.max_pool3d(t1, 2, 2)                                        # 32 x 64 x 64 x 64
                                    
        t2 = self.encoder2(out)                                             # 64 x 64 x 64 x 64
        out = F.max_pool3d(t2, 2, 2)                                        # 64 x 32 x 32 x 32
        
        t3 = self.encoder3(out)                                             # 128 x 32 x 32 x 32
        out = F.max_pool3d(t3, 2, 2)                                        # 128 x 16 x 16 x 16
        
        t4 = self.encoder4(out)                                             # 256 x 16 x 16 x 16
        out = F.max_pool3d(t4, 2, 2)                                        # 256 x 8 x 8 x 8
        
        out = self.encoder5(out)                                            # 512 x 8 x 8 x 8
        
        
        
        out = self.conv_trans1(out)                                         # 256 x 16 x 16 x 16
        out = self.decoder1(torch.cat([out, t4], dim=1))                    # 256 x 16 x 16 x 16
        
        out = self.conv_trans2(out)                                          # 128 x 32 x 32 x 32
        out = self.decoder2(torch.cat([out, t3], dim=1))                    # 128 x 32 x 32 x 32
        
        out = self.conv_trans3(out)                                         # 64 x 64 x 64 x 64
        out = self.decoder3(torch.cat([out, t2], dim=1))                    # 64 x 64 x 64 x 64                

        out = self.conv_trans4(out)                                         # 32 x 128 x 128 x 128
        out = self.decoder4(torch.cat([out, t1], dim=1))                    # 32 x 128 x 128 x 128

        out = self.out_conv(out)                                            # out_channels x 128 x 128
        
        out = self.soft(out)                                             # softmax
        return out

In [15]:
from torchsummary import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet3d_bn(in_channels=4, out_channels=4)
input_tensor = torch.randn([1, 4, 128, 128, 128]).float()

model.to(device)
input_tensor = input_tensor.to(device)


out = model(input_tensor)
print(out.shape)
summary(model, (4, 128, 128, 128))

torch.Size([1, 4, 128, 128, 128])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 32, 128, 128, 128]           3,488
            Conv3d-2       [-1, 64, 64, 64, 64]          55,360
            Conv3d-3      [-1, 128, 32, 32, 32]         221,312
            Conv3d-4      [-1, 256, 16, 16, 16]         884,992
            Conv3d-5         [-1, 512, 8, 8, 8]       3,539,456
   ConvTranspose3d-6      [-1, 256, 16, 16, 16]       1,048,832
            Conv3d-7      [-1, 256, 16, 16, 16]       3,539,200
   ConvTranspose3d-8      [-1, 128, 32, 32, 32]         262,272
            Conv3d-9      [-1, 128, 32, 32, 32]         884,864
  ConvTranspose3d-10       [-1, 64, 64, 64, 64]          65,600
           Conv3d-11       [-1, 64, 64, 64, 64]         221,248
  ConvTranspose3d-12    [-1, 32, 128, 128, 128]          16,416
           Conv3d-13    [-1, 32, 128, 128, 128]          55,328
     

In [32]:
# simple UNet3d_ln
class UNet3d_ln(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet3d_ln, self).__init__()
        self.encoder1 = nn.Conv3d(in_channels, 32, kernel_size=3, padding=1)
        self.encoder2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.encoder3 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.encoder4 = nn.Conv3d(128, 256, kernel_size=3, padding=1)
        self.encoder5 = nn.Conv3d(256, 512, kernel_size=3, padding=1)

        self.decoder1 = nn.Conv3d(512, 256, kernel_size=3, padding=1)
        self.conv_trans1 = nn.ConvTranspose3d(512, 256, kernel_size=2, stride=2)
        self.decoder2 = nn.Conv3d(256, 128, kernel_size=3, padding=1)
        self.conv_trans2 = nn.ConvTranspose3d(256, 128, kernel_size=2, stride=2)
        self.decoder3 = nn.Conv3d(128, 64, kernel_size=3, padding=1)
        self.conv_trans3 = nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2)
        self.decoder4 = nn.Conv3d(64, 32, kernel_size=3, padding=1)
        self.conv_trans4 = nn.ConvTranspose3d(64, 32, kernel_size=2, stride=2)
        self.out_conv = nn.Conv3d(32, out_channels, kernel_size=3, padding=1)

        self.soft = nn.Softmax(dim=1)
        
        
    def forward(self, x):
        # 编码器
        out = self.encoder1(x)                                              # 32 x 128 x 128 x 128
        out = F.relu(F.layer_norm(out, out.shape[-3:]))                     
        t1 = out                                                            # 32 x 128 x 128 x 128
        
        out = F.max_pool3d(t1, 2, 2)                                        # 32 x 64 x 64 x 64
        out = self.encoder2(out)                                            # 64 x 64 x 64 x 64
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        t2 = out                                                            # 64 x 64 x 64 x 64

        out = F.max_pool3d(t2, 2, 2)                                        # 64 x 32 x 32 x 32
        out = self.encoder3(out)                                            # 128 x 32 x 32 x 32
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        t3 = out                                                            # 128 x 32 x 32 x 32

        out = F.max_pool3d(t3, 2, 2)                                        # 128 x 16 x 16 x 16
        out = self.encoder4(out)                                            # 256 x 16 x 16 x 16
        out = F.relu(F.layer_norm(out, out.shape[-3:]))                     
        t4 = out                                                            # 256 x 16 x 16 x 16
        
        out = F.max_pool3d(t4, 2, 2)                                        # 256 x 8 x 8 x 8
        out = self.encoder5(out)                                            # 512 x 8 x 8 x 8
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        
        # 解码器
        out = self.conv_trans1(out)                                         # 256 x 16 x 16 x 16
        out = self.decoder1(torch.cat([out, t4], dim=1))                    # 256 x 16 x 16 x 16
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        
        out = self.conv_trans2(out)                                         # 128 x 32 x 32 x 32
        out = self.decoder2(torch.cat([out, t3], dim=1))                    # 128 x 32 x 32 x 32
        out = F.relu(F.layer_norm(out, out.shape[-3:]))

        out = self.conv_trans3(out)                                         # 64 x 64 x 64 x 64
        out = self.decoder3(torch.cat([out, t2], dim=1))                    # 64 x 64 x 64 x 64
        out = F.relu(F.layer_norm(out, out.shape[-3:]))                 
        
        out = self.conv_trans4(out)                                         # 32 x 128 x 128 x 128
        out = self.decoder4(torch.cat([out, t1], dim=1))                    # 32 x 128 x 128 x 128
        out = F.relu(F.layer_norm(out, out.shape[-3:]))                     
        
        out = self.out_conv(out)                                            # out_channels x 128 x 128 x 128
        
        out = self.soft(out)                                                # softmax
        
        return out

In [35]:

# 改进
class UNet3d_ln_double(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet3d_ln_double, self).__init__()
        self.encoder1 = nn.Conv3d(in_channels, 32, kernel_size=3, padding=1)
        self.encoder2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.encoder3 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.encoder4 = nn.Conv3d(128, 256, kernel_size=3, padding=1)
        self.encoder5 = nn.Conv3d(256, 512, kernel_size=3, padding=1)

        self.conv_32    = nn.Conv3d(32, 32, kernel_size=3, padding=1)
        self.conv_64    = nn.Conv3d(64, 64, kernel_size=3, padding=1)
        self.conv_128    = nn.Conv3d(128, 128, kernel_size=3, padding=1)
        self.conv_256    = nn.Conv3d(256, 256, kernel_size=3, padding=1)
        self.conv_512    = nn.Conv3d(512, 512, kernel_size=3, padding=1)    
        
        self.decoder1 = nn.Conv3d(512, 256, kernel_size=3, padding=1)
        self.conv_trans1 = nn.ConvTranspose3d(512, 256, kernel_size=2, stride=2)
        self.decoder2 = nn.Conv3d(256, 128, kernel_size=3, padding=1)
        self.conv_trans2 = nn.ConvTranspose3d(256, 128, kernel_size=2, stride=2)
        self.decoder3 = nn.Conv3d(128, 64, kernel_size=3, padding=1)
        self.conv_trans3 = nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2)
        self.decoder4 = nn.Conv3d(64, 32, kernel_size=3, padding=1)
        self.conv_trans4 = nn.ConvTranspose3d(64, 32, kernel_size=2, stride=2)
        self.out_conv = nn.Conv3d(32, out_channels, kernel_size=3, padding=1)

        self.soft = nn.Softmax(dim=1)
        
        
        
    def forward(self, x):
        # 编码器
        out = self.encoder1(x)                                              # 32 x 128 x 128 x 128
        out = F.relu(F.layer_norm(out, out.shape[-3:]))                     
        out = self.conv_32(out)
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        t1 = out                                                            # 32 x 128 x 128 x 128
        
        out = F.max_pool3d(t1, 2, 2)                                        # 32 x 64 x 64 x 64
        out = self.encoder2(out)                                            # 64 x 64 x 64 x 64
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        out = self.conv_64(out)
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        t2 = out                                                            # 64 x 64 x 64 x 64

        out = F.max_pool3d(t2, 2, 2)                                        # 64 x 32 x 32 x 32
        out = self.encoder3(out)                                            # 128 x 32 x 32 x 32
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        out = self.conv_128(out)
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        t3 = out                                                            # 128 x 32 x 32 x 32

        out = F.max_pool3d(t3, 2, 2)                                        # 128 x 16 x 16 x 16
        out = self.encoder4(out)                                            # 256 x 16 x 16 x 16
        out = F.relu(F.layer_norm(out, out.shape[-3:]))                     
        out = self.conv_256(out)
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        t4 = out                                                            # 256 x 16 x 16 x 16
        
        out = F.max_pool3d(t4, 2, 2)                                        # 256 x 8 x 8 x 8
        out = self.encoder5(out)                                            # 512 x 8 x 8 x 8
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        out = self.conv_512(out)
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        
        # 解码器
        out = self.conv_trans1(out)                                         # 256 x 16 x 16 x 16
        out = self.decoder1(torch.cat([out, t4], dim=1))                    # 256 x 16 x 16 x 16
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        out = self.conv_256(out)
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        
        out = self.conv_trans2(out)                                         # 128 x 32 x 32 x 32
        out = self.decoder2(torch.cat([out, t3], dim=1))                    # 128 x 32 x 32 x 32
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        out = self.conv_128(out)
        out = F.relu(F.layer_norm(out, out.shape[-3:]))

        out = self.conv_trans3(out)                                         # 64 x 64 x 64 x 64
        out = self.decoder3(torch.cat([out, t2], dim=1))                    # 64 x 64 x 64 x 64
        out = F.relu(F.layer_norm(out, out.shape[-3:]))                 
        out = self.conv_64(out)
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        
        out = self.conv_trans4(out)                                         # 32 x 128 x 128 x 128
        out = self.decoder4(torch.cat([out, t1], dim=1))                    # 32 x 128 x 128 x 128
        out = F.relu(F.layer_norm(out, out.shape[-3:]))                     
        out = self.conv_32(out)
        out = F.relu(F.layer_norm(out, out.shape[-3:]))
        
        out = self.out_conv(out)                                            # out_channels x 128 x 128 x 128
        
        out = self.soft(out)                                                # softmax
        
        return out

In [34]:
from torchsummary import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet3d_ln(in_channels=4, out_channels=4)
input_tensor = torch.randn([1, 4, 128, 128, 128]).float()

model.to(device)
input_tensor = input_tensor.to(device)


out = model(input_tensor)
print(out.shape)
summary(model, (4, 128, 128, 128))

torch.Size([1, 4, 128, 128, 128])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 32, 128, 128, 128]           3,488
            Conv3d-2       [-1, 64, 64, 64, 64]          55,360
            Conv3d-3      [-1, 128, 32, 32, 32]         221,312
            Conv3d-4      [-1, 256, 16, 16, 16]         884,992
            Conv3d-5         [-1, 512, 8, 8, 8]       3,539,456
   ConvTranspose3d-6      [-1, 256, 16, 16, 16]       1,048,832
            Conv3d-7      [-1, 256, 16, 16, 16]       3,539,200
   ConvTranspose3d-8      [-1, 128, 32, 32, 32]         262,272
            Conv3d-9      [-1, 128, 32, 32, 32]         884,864
  ConvTranspose3d-10       [-1, 64, 64, 64, 64]          65,600
           Conv3d-11       [-1, 64, 64, 64, 64]         221,248
  ConvTranspose3d-12    [-1, 32, 128, 128, 128]          16,416
           Conv3d-13    [-1, 32, 128, 128, 128]          55,328
     

In [36]:
from torchsummary import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet3d_ln_double(in_channels=4, out_channels=4)
input_tensor = torch.randn([1, 4, 128, 128, 128]).float()

model.to(device)
input_tensor = input_tensor.to(device)


out = model(input_tensor)
print(out.shape)
summary(model, (4, 128, 128, 128))

torch.Size([1, 4, 128, 128, 128])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 32, 128, 128, 128]           3,488
            Conv3d-2    [-1, 32, 128, 128, 128]          27,680
            Conv3d-3       [-1, 64, 64, 64, 64]          55,360
            Conv3d-4       [-1, 64, 64, 64, 64]         110,656
            Conv3d-5      [-1, 128, 32, 32, 32]         221,312
            Conv3d-6      [-1, 128, 32, 32, 32]         442,496
            Conv3d-7      [-1, 256, 16, 16, 16]         884,992
            Conv3d-8      [-1, 256, 16, 16, 16]       1,769,728
            Conv3d-9         [-1, 512, 8, 8, 8]       3,539,456
           Conv3d-10         [-1, 512, 8, 8, 8]       7,078,400
  ConvTranspose3d-11      [-1, 256, 16, 16, 16]       1,048,832
           Conv3d-12      [-1, 256, 16, 16, 16]       3,539,200
           Conv3d-13      [-1, 256, 16, 16, 16]       1,769,728
  Con

In [4]:
import os
import datetime
import tkinter as tk

def create_experiment_dir():
    # 获取当前时间
    current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    
    # 创建results文件夹
    results_dir = "results"
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    # 创建实验结果目录
    experiment_dir = os.path.join(results_dir, current_time)
    os.makedirs(experiment_dir)
    
    return experiment_dir

def write_readme(experiment_dir, experiment_content):
    # 创建README.md文件
    readme_file = os.path.join(experiment_dir, "README.md")
    with open(readme_file, "w") as f:
        f.write(experiment_content)

def get_experiment_content():
    # 创建弹窗
    windows = tk.Tk()
    windows.title("Experiment Content")
    
    # 创建输入框
    label = tk.Label(windows, text="input experiment content:")
    label.pack()
    entry = tk.Text(windows, height=10, width=40)
    entry.pack()
    
    # 创建确定按钮
    def confirm():
        experiment_content = entry.get("1.0", tk.END)
        windows.destroy()
        return experiment_content
    
    button = tk.Button(windows, text="confirm", command=lambda: confirm())
    button.pack()
    
    # 显示弹窗
    windows.mainloop()
    
    return confirm()

def main():
    # 获取实验内容
    experiment_content = get_experiment_content()
    
    # 创建实验结果目录
    experiment_dir = create_experiment_dir()
    
    # 写入README.md文件
    write_readme(experiment_dir, experiment_content)

if __name__ == "__main__":
    main()

TclError: invalid command name ".!text"